## Embedding from OpenAI:

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv(dotenv_path=f"{os.path.abspath('')}/../../backend/.env.dev", verbose=True)

True

In [3]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [4]:
embedding.embed_query("Chúng tôi là những nghiên cứu viên.")

[0.03680105493052272,
 -0.015048128494173396,
 -0.027685541551852235,
 -0.0353696940952956,
 0.0344468428344102,
 0.005598317809968212,
 -0.015340050622727995,
 0.026499019832603252,
 -0.03520018966208493,
 -0.06685963992087292,
 -0.039588443780854646,
 0.015434219441848623,
 -0.0463309062704466,
 -0.02211076571383354,
 0.0017750764895069337,
 0.006897843416013545,
 -0.02642368514983578,
 -0.03520018966208493,
 0.005603026576887143,
 0.024069471191078114,
 0.016008647096442532,
 -0.026329515399392583,
 0.043091509323509844,
 -0.027478372571225543,
 0.03237513440169184,
 -0.026273013921655694,
 0.01652657420462212,
 0.0029333492161172916,
 -0.020754739561373886,
 0.011187219017421132,
 0.028307054081667744,
 -0.01769426271884052,
 0.012411409940698993,
 0.06712331596717193,
 -0.017713097786516245,
 0.08874440158640404,
 0.01838169426023078,
 -0.012656248125354564,
 -0.03800640985480229,
 -0.036236043878444105,
 0.002072884460294876,
 0.006751882351736245,
 -0.03307197975162968,
 0.02651

## Embedding from PhoBERT:

- https://gist.github.com/albahnsen/b02d2183c93067e3f248a428430c970e
- https://github.com/VinAIResearch/PhoBERT
- https://www.geeksforgeeks.org/how-to-generate-word-embedding-using-bert/
- https://lukesalamone.github.io/posts/what-are-attention-masks/

In [7]:
import torch
from transformers import AutoModel, AutoTokenizer

In [8]:
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

phobert_v2 = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer_v2 = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

/home/knguyen02311/miniconda3/envs/aienv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
phobert, tokenizer

(RobertaModel(
   (embeddings): RobertaEmbeddings(
     (word_embeddings): Embedding(64001, 768, padding_idx=1)
     (position_embeddings): Embedding(258, 768, padding_idx=1)
     (token_type_embeddings): Embedding(1, 768)
     (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
     (dropout): Dropout(p=0.1, inplace=False)
   )
   (encoder): RobertaEncoder(
     (layer): ModuleList(
       (0-11): 12 x RobertaLayer(
         (attention): RobertaAttention(
           (self): RobertaSelfAttention(
             (query): Linear(in_features=768, out_features=768, bias=True)
             (key): Linear(in_features=768, out_features=768, bias=True)
             (value): Linear(in_features=768, out_features=768, bias=True)
             (dropout): Dropout(p=0.1, inplace=False)
           )
           (output): RobertaSelfOutput(
             (dense): Linear(in_features=768, out_features=768, bias=True)
             (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True

In [123]:
# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'

# special token is added when encode
input_ids = torch.tensor([tokenizer_v2.encode(sentence)])
print(input_ids)

tensor([[    0,   746,     8,    21, 46349,     5,     2]])


In [126]:
with torch.no_grad():
    features = phobert_v2(input_ids)  # Models outputs are now tuples
sentence_embedding = features.last_hidden_state.mean(dim=1)
sentence_embedding[0]

tensor([ 1.3425e-01,  5.9045e-02, -4.1865e-02, -1.4196e-01,  1.3992e-01,
         2.0046e-01, -1.3856e-01,  5.0560e-02, -8.7782e-02,  4.3882e-02,
         6.5798e-02,  2.9922e-01, -7.6993e-02,  1.4478e-01, -8.6979e-02,
        -7.4493e-02, -3.7784e-01,  7.7734e-02, -3.0619e-01, -1.1001e-01,
        -1.0378e-01, -2.7450e-01,  2.4107e-01, -1.6149e-01,  7.4696e-02,
        -2.2744e-01,  3.9212e-01, -1.5903e-03,  1.1093e-01,  1.4273e-01,
        -1.0417e-01, -1.9264e-01, -1.4071e-01,  1.2377e-02,  2.4560e-01,
         5.3694e-01, -1.5574e-01,  1.0420e-01, -1.0250e-01,  4.5808e-01,
         3.5582e-01,  2.1015e-02,  3.6090e-01, -1.5405e-01,  3.8498e-01,
        -3.3508e-02,  1.9254e-01,  1.8713e-01,  3.0987e-01,  1.6441e-01,
        -4.1546e-02, -1.1495e-01, -1.1851e-01, -3.6544e-01,  8.8668e-01,
        -2.7447e-01, -6.4167e-02, -8.6043e-02, -6.0417e-02,  7.2270e-02,
        -3.7088e-02, -4.2211e-02,  6.8692e-02, -1.4098e-01,  1.1571e-01,
        -9.2889e-02,  9.5089e-02,  2.2534e-01,  1.0

## Word segmentation for unformatted text

In [26]:
import py_vncorenlp

In [27]:
vncorenlp_segmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir="../VnCoreNLP/")

2024-05-24 16:43:19 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [5]:
vncorenlp_segmenter.annotate_file(input_file="./testdata_quang_binh.txt", output_file="./testdata_quang_binh_segmented.txt")

2024-05-24 10:28:56 INFO  Annotation:100 - Start processing ./testdata_quang_binh.txt
2024-05-24 10:28:57 INFO  Annotation:113 - Wrote output to ./testdata_quang_binh_segmented.txt


In [81]:
# manual way
with open("./testdata_quang_binh_segmented_manual.txt", "a") as segmented_file:
    file = open("./testdata_quang_binh.txt", "r")
    for line in file:
        if line == "\n":
            segmented_file.write("\n")
            continue
        segmented_list = vncorenlp_segmenter.word_segment(line)
        for i in segmented_list:
            segmented_file.write(i + " ")
        segmented_file.write("\n")
    file.close()

## Embedding to vector DB

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

qdrant_client = QdrantClient(":memory:")

In [22]:
# qdrant_client.delete_collection(collection_name="test_coll_phobert")
# qdrant_client.delete_collection(collection_name="test_coll_phobert_v2")
# qdrant_client.delete_collection(collection_name="test_coll_openai_embed")

if not qdrant_client.collection_exists(collection_name="test_coll_phobert"):
    qdrant_client.create_collection(
        collection_name="test_coll_phobert",
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )
    
if not qdrant_client.collection_exists(collection_name="test_coll_phobert_v2"):
    qdrant_client.create_collection(
        collection_name="test_coll_phobert_v2",
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )
    
if not qdrant_client.collection_exists(collection_name="test_coll_openai_embed"):
    qdrant_client.create_collection(
        collection_name="test_coll_openai_embed",
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    )

In [13]:
with open("./testdata_quang_binh_segmented_manual.txt", "r") as file:
    testdata = file.read()
with open("./testdata_quang_binh.txt", "r") as file:
    testdata_unsegmented = file.read()
testdata

'Giới_thiệu Quảng_Bình : \nQuảng_Bình là một tỉnh ven biển phía nam vùng Bắc_Trung_Bộ , miền Trung của Việt_Nam , phía Nam giáp với Quảng_Trị , phía Bắc giáp Hà_Tĩnh , phía Đông giáp Biển Đông , phía Tây giáp Lào . Tỉnh_lỵ của tỉnh là thành_phố Đồng_Hới . \nTỉnh Quảng_Bình có 8 đơn_vị hành_chính cấp huyện ( 6 huyện , 1 thành_phố và 1 thị_xã ) , 159 đơn_vị hành_chính cấp xã ( 136 xã , 16 phường và 7 thị_trấn ) . \nTuy diện_tích không quá lớn , song Quảng_Bình lại là vùng_đất tập_trung nhiều thắng_cảnh đẹp nổi_tiếng trong và ngoài nước . Trong đó phải kể đến “ vương_quốc hang_động ” gồm 404 hang_động lớn_nhỏ , những bãi biển hoang_sơ tuyệt đẹp và những điểm đến tâm_linh hấp_dẫn . \n\nPhương_tiện di_chuyển đến Quảng_Bình : \nĐến du_lịch Quảng_Bình , bạn có_thể lựa_chọn phương_tiện di_chuyển tàu_hoả , ô_tô , máy_bay hoặc tự_túc bằng xe cá_nhân . \nTàu_hoả : \nĐi từ Hà_Nội : tàu đi từ Hà_Nội - Quảng_Bình mất khoảng 10 tiếng . Giá vé ngồi khoảng 300.000 VNĐ , vé giường_nằm khoảng từ 500.000 

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", ".", " "]
)

splited_data = text_splitter.create_documents([testdata])
splited_data_unsegmented = text_splitter.create_documents([testdata_unsegmented])

In [15]:
splited_data[0].page_content.replace("\n", "")

'Giới_thiệu Quảng_Bình : Quảng_Bình là một tỉnh ven biển phía nam vùng Bắc_Trung_Bộ , miền Trung của Việt_Nam , phía Nam giáp với Quảng_Trị , phía Bắc giáp Hà_Tĩnh , phía Đông giáp Biển Đông , phía Tây giáp Lào . Tỉnh_lỵ của tỉnh là thành_phố Đồng_Hới . Tỉnh Quảng_Bình có 8 đơn_vị hành_chính cấp huyện ( 6 huyện , 1 thành_phố và 1 thị_xã ) , 159 đơn_vị hành_chính cấp xã ( 136 xã , 16 phường và 7 thị_trấn ) . Tuy diện_tích không quá lớn , song Quảng_Bình lại là vùng_đất tập_trung nhiều thắng_cảnh đẹp nổi_tiếng trong và ngoài nước . Trong đó phải kể đến “ vương_quốc hang_động ” gồm 404 hang_động lớn_nhỏ , những bãi biển hoang_sơ tuyệt đẹp và những điểm đến tâm_linh hấp_dẫn .'

In [18]:
# for phobert v1 & v2
sentence_embeddings_phobert = []
sentence_embeddings_phobert_v2 = []

print("starting tokenizer for phoBERT")
for i, document in enumerate(splited_data):
    print(i)
    input_ids = torch.tensor([tokenizer.encode(document.page_content.replace("\n", ""))])
    input_ids_v2 = torch.tensor([tokenizer_v2.encode(document.page_content.replace("\n", ""))])
    with torch.no_grad():
        features = phobert(input_ids)
        features_v2 = phobert_v2(input_ids_v2)
    sentence_embeddings_phobert.append(features.last_hidden_state.mean(dim=1)[0])
    sentence_embeddings_phobert_v2.append(features_v2.last_hidden_state.mean(dim=1)[0])
    
# for OpenAI embedding
sentence_embeddings_openai = []

print("starting tokenizer for OpenAI embedding model")
for i, document in enumerate(splited_data_unsegmented):
    print(i)
    sentence_embeddings_openai.append(embedding.embed_query(document.page_content.replace("\n", "")))

starting tokenizer for phoBERT
0


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
starting tokenizer for OpenAI embedding model
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [19]:
len(sentence_embeddings_phobert), len(sentence_embeddings_phobert_v2), len(sentence_embeddings_openai), len(splited_data)

(57, 57, 54, 57)

In [24]:
# insert vectors to DB
operation_info = qdrant_client.upsert(
    collection_name="test_coll_phobert",
    points=[
        PointStruct(id=i, vector=embedding.tolist())
        for i, embedding in enumerate(sentence_embeddings_phobert)
    ],
)

operation_info_v2 = qdrant_client.upsert(
    collection_name="test_coll_phobert_v2",
    points=[
        PointStruct(id=i, vector=embedding.tolist())
        for i, embedding in enumerate(sentence_embeddings_phobert_v2)
    ],
)

operation_info_openai = qdrant_client.upsert(
    collection_name="test_coll_openai_embed",
    points=[
        PointStruct(id=i, vector=embedding)
        for i, embedding in enumerate(sentence_embeddings_openai)
    ],
)

operation_info, operation_info_v2, operation_info_openai

(UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>),
 UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>),
 UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>))

In [28]:
# test with a query
search_query = "địa điểm du lịch tham quan ở quảng bình"
segmented_query = vncorenlp_segmenter.word_segment(search_query)
input_ids = torch.tensor([tokenizer.encode(segmented_query)])
input_ids_v2 = torch.tensor([tokenizer_v2.encode(segmented_query)])
with torch.no_grad():
    features_phobert = phobert(input_ids)
    features_phobert_v2 = phobert_v2(input_ids)
search_embedding_phobert = features_phobert.last_hidden_state.mean(dim=1)[0].tolist()
search_embedding_phobert_v2 = features_phobert_v2.last_hidden_state.mean(dim=1)[0].tolist()
search_embedding_openai = OpenAIEmbeddings(model="text-embedding-3-small").embed_query(search_query)

search_result_phobert = qdrant_client.search(
    collection_name="test_coll_phobert",
    query_vector=search_embedding_phobert,
    limit=5
)

search_result_phobert_v2 = qdrant_client.search(
    collection_name="test_coll_phobert_v2",
    query_vector=search_embedding_phobert_v2,
    limit=5
)


search_result_openai = qdrant_client.search(
    collection_name="test_coll_openai_embed",
    query_vector=search_embedding_openai,
    limit=5
)

In [29]:
for point in search_result_phobert:
    print(point.id, point.score)
    print(splited_data[point.id].page_content)
    print('')

44 0.35412369773827557
Các quán ăn ngon ở Quảng_Bình : 
1 . Quán đặc_sản lươn đồng Chị Ngạnh 
Địa_chỉ : số 18 , đường Hoàng_Diệu , thành_phố Đồng_Hới , tỉnh Quảng_Bình 
Giờ mở_cửa tham_khảo : 6:00 - 12:00 , 16:00 - 22:00 
Giá tham_khảo : 30.000 - 100.000 VNĐ / món 
Nhắc đến đặc_sản Quảng_Bình lươn đồng không_thể_nào bỏ_qua quán Chị Ngạnh trên đường Hoàng_Diệu . Đây là quán ngon rất nổi_tiếng với các món lươn đậm_đà như cháo lươn , miến lươn , súp lươn , ... Lươn tại đây dai , khi ăn có vị ngọt , mềm hoà_quyện cùng vị cay của ớt , tiêu và màu vàng của nghệ . Tất_cả hoà_quyện với nhau tạo nên món ngon khó cưỡng khiến thực_khách không_thể_nào quên . 
2 . Quán hải_sản Mệ_Toại 
Địa_chỉ : số 185 , đường Trương_Pháp , thành_phố Đồng_Hới , tỉnh Quảng_Bình 
Giờ mở_cửa tham_khảo : 5:00 - 20:00

48 0.34547559939259376
Giá tham_khảo : 20.000 - 30.000 VNĐ / món 
Cháo canh Gia_Bảo là quán ăn ngon ở Quảng_Bình hoạt_động lâu_đời và là địa_chỉ tin_cậy của nhiều thực_khách . Quán sở_hữu thực_đơn cháo ca

In [30]:
for point in search_result_phobert_v2:
    print(point.id, point.score)
    print(splited_data[point.id].page_content)
    print('')

7 0.3772608150322223
Các điểm du_lịch ở Quảng_Bình :

9 0.20297762341140416
. Càng đi_sâu vào trong , một thế_giới huyền_ảo dần hiện ra .

18 0.19904468741014664
Thời_điểm du_lịch Quảng_Bình : 
Quảng_Bình mang tính_chất khí_hậu nhiệt_đới gió_mùa , có sự ảnh_hưởng khí_hậu chuyển_tiếp giữa miền Bắc và miền Nam , do_đó có hai mùa rõ_rệt là mùa mưa và mùa khô . Mùa mưa từ tháng 9 đến tháng 3 năm sau , lượng mưa trung_bình hàng năm 2.000 – 2.300 mm / năm . Mùa khô từ tháng 4 đến tháng 8 với nhiệt_độ trung_bình 24 – 25 độ C. Ba tháng có nhiệt_độ cao nhất là tháng 6 , 7 và 8 . Du_lịch Quảng_Bình bạn nên xem trước dự_báo thời_tiết để tránh tình_trạng mưa_gió làm ảnh_hưởng đến chuyến đi .

0 0.19793447279486176
Giới_thiệu Quảng_Bình : 
Quảng_Bình là một tỉnh ven biển phía nam vùng Bắc_Trung_Bộ , miền Trung của Việt_Nam , phía Nam giáp với Quảng_Trị , phía Bắc giáp Hà_Tĩnh , phía Đông giáp Biển Đông , phía Tây giáp Lào . Tỉnh_lỵ của tỉnh là thành_phố Đồng_Hới . 
Tỉnh Quảng_Bình có 8 đơn_vị hành_

In [31]:
for point in search_result_openai:
    print(point.id, point.score)
    print(splited_data_unsegmented[point.id].page_content)
    print('')

7 0.6756403397383883
Các điểm du lịch ở Quảng Bình:

29 0.5503828089234761
Gợi ý lịch trình du lịch Quảng Bình 4 ngày 3 đêm thú vị:
Với rất nhiều điểm đến hấp dẫn, bạn cần lên kế hoạch và có một lịch trình du lịch Quảng Bình 4 ngày 3 đêm thật cụ thể. Điều này sẽ giúp bạn tiết kiệm khá nhiều thời gian, công sức di chuyển và có thể tham quan trọn vẹn được nhiều nơi. Lịch trình sau đây là một gợi ý mà bạn có thể tham khảo:
Ngày 1: Check-in khách sạn - Vũng Chùa Đảo Yến - Đồng Hới
Di chuyển đến Đồng Hới, check-in nhận phòng khách sạn và dành một ít thời gian nghỉ ngơi sau hành trình dài.
Đến Vũng Chùa, Đảo Yến thăm lăng mộ Đại tướng Võ Nguyên Giáp, thắp nén nhang tưởng nhớ đến vị anh hùng dân tộc và tham quan chụp ảnh lưu niệm.
Về khách sạn ăn tối, khám phá thành phố Quảng Bình về đêm, đến thăm tượng đài mẹ Suốt, Quảng Bình Quan…

32 0.5412181950621962
Buổi sáng của ngày cuối cùng trong lịch trình du lịch Quảng Bình 4 ngày 3 đêm, bạn hãy dành thời gian để dạo quanh thành phố Đồng Hới và đế